<a href="https://colab.research.google.com/github/swathi942/earthquake-streamlit-dashboard/blob/main/EarthQuake_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas

In [ ]:
import requests
import pandas as pd
import re
from datetime import datetime


In [ ]:
import requests
import pandas as pd
from datetime import datetime

url ="https://earthquake.usgs.gov/fdsnws/event/1/query"

all_records=[]
start_year = datetime.now().year - 5 # Last 5 years
end_year = datetime.now().year

for year in range(start_year,end_year + 1):
    for month in range(1,13):
      start_date=f"{year}-{month:02d}-01"
      if month == 12:
        end_date =f"{year + 1}-01-01"
      else:
        end_date =f"{year}-{month+1:02d}-01"

      params = {
          "format": "geojson",
          "starttime": start_date,
          "endtime": end_date,
          "minmagnitude": 3
      }

      response = requests.get(url,params=params)
      if response.status_code !=200:
        print(f" Failed for {start_date}:{response.text[:200]}")
        continue

      try:
          data = response.json()
      except Exception as e:
          print(f" JSON error for {start_date}:{e}")
          continue

      for f in data["features"]:
          p =f["properties"]
          g =f["geometry"]["coordinates"]

          all_records.append({
              "id": f.get("id"),
              "time": pd.to_datetime(p.get("time"),unit="ms"),
              "updated": pd.to_datetime(p.get("updated"),unit="ms"),
              "latitude": g[1] if g else None,
              "longitude": g[0] if g else None,
              "depth_km": g[2] if g else None,
              "mag": p.get("mag"),
              "magType": p.get("magType"),
              "alert": p.get("alert"),
              "felt": p.get("felt"),
              "cdi": p.get("cdi"),
              "mmi": p.get("mmi"),
              "code": p.get("code"),
              "place":p.get("place"),
              "status":p.get("status"),
              "tsunami":p.get("tsunami"),
              "sig":p.get("sig"),
              "net":p.get("net"),
              "nst":p.get("nst"),
              "dmin":p.get("dmin"),
              "rms":p.get("rms"),
              "gap":p.get("gap"),
              "types":p.get("types"),
              "ids":p.get("ids"),
              "sources":p.get("sources"),
              "type":p.get("type"),










          })

df = pd.DataFrame(all_records)


print("Rows:",df.shape[0])
print("Columns:",df.shape[1])
print(df.head())

Rows: 105997
Columns: 26
           id                    time                 updated  latitude  \
0  us6000ddi8 2021-01-31 23:20:49.923 2021-04-16 19:02:44.040  -31.7493   
1  us6000dev6 2021-01-31 23:08:17.161 2021-04-16 19:03:47.040  -15.4902   
2  us6000dev5 2021-01-31 22:54:19.760 2021-04-16 19:03:47.040   19.7529   
3  us6000ddhs 2021-01-31 22:06:00.832 2021-04-16 19:02:43.040   28.1524   
4  us6000dev4 2021-01-31 21:51:14.016 2021-04-16 19:03:46.040   71.3212   

   longitude  depth_km  mag magType alert  felt  ...  sig  net nst   dmin  \
0   -68.9337     17.27  4.7     mwr  None  12.0  ...  344   us NaN  0.294   
1  -177.2052    426.71  4.1      mb  None   NaN  ...  259   us NaN  1.471   
2   121.3159     46.73  4.7      mb  None   NaN  ...  340   us NaN  3.057   
3    57.2570     10.00  4.9      mb  None   NaN  ...  369   us NaN  3.330   
4    -3.7578     10.00  4.0      mb  None   NaN  ...  246   us NaN  6.023   

    rms    gap                                   types       

In [ ]:
df.columns

Index(['id', 'time', 'updated', 'latitude', 'longitude', 'depth_km', 'mag',
       'magType', 'alert', 'felt', 'cdi', 'mmi', 'code', 'place', 'status',
       'tsunami', 'sig', 'net', 'nst', 'dmin', 'rms', 'gap', 'types', 'ids',
       'sources', 'type', 'country', 'year', 'month', 'day', 'day_of_week',
       'depth_category', 'magnitude_category'],
      dtype='object')

In [ ]:
def extract_country(place):
    if pd.isna(place):
        return "Unknown"
    match = re.search(r",\s*([^,]+)$", place)
    return match.group(1) if match else "Unknown"

df["country"] = df["place"].apply(extract_country)

In [ ]:
# Year, Month, Day, Day of Week
df["year"] = df["time"].dt.year
df["month"] = df["time"].dt.month
df["day"] = df["time"].dt.day
df["day_of_week"] = df["time"].dt.day_name()

df[["time", "year", "month", "day", "day_of_week"]].head()


,time,year,month,day,day_of_week
0,2021-01-31 23:20:49.923,2021,1,31,Sunday
1,2021-01-31 23:08:17.161,2021,1,31,Sunday
2,2021-01-31 22:54:19.760,2021,1,31,Sunday
3,2021-01-31 22:06:00.832,2021,1,31,Sunday
4,2021-01-31 21:51:14.016,2021,1,31,Sunday


In [ ]:
def depth_flag(depth):
    if depth < 70:
        return "Shallow"
    elif depth <= 300:
        return "Intermediate"
    else:
        return "Deep"

df["depth_category"] = df["depth_km"].apply(depth_flag)

df["depth_category"].value_counts()


,count
depth_category,
Shallow,79109
Intermediate,20612
Deep,6276


In [ ]:
def magnitude_flag(mag):
    if mag >= 7:
        return "Strong / Destructive"
    elif mag >= 5:
        return "Moderate"
    else:
        return "Mild"

df["magnitude_category"] = df["mag"].apply(magnitude_flag)

df["magnitude_category"].value_counts()


,count
magnitude_category,
Mild,96417
Moderate,9505
Strong / Destructive,75


In [ ]:
df[[
    "time",
    "depth_km",
    "mag",
    "depth_category",
    "magnitude_category"
]].head(10)


,time,depth_km,mag,depth_category,magnitude_category
0,2021-01-31 23:20:49.923,17.27,4.70,Shallow,Mild
1,2021-01-31 23:08:17.161,426.71,4.10,Deep,Mild
2,2021-01-31 22:54:19.760,46.73,4.70,Shallow,Mild
3,2021-01-31 22:06:00.832,10.00,4.90,Shallow,Mild
4,2021-01-31 21:51:14.016,10.00,4.00,Shallow,Mild
5,2021-01-31 21:34:54.690,46.00,3.55,Shallow,Mild
6,2021-01-31 21:26:30.180,30.00,3.27,Shallow,Mild
7,2021-01-31 21:14:31.020,28.00,3.48,Shallow,Mild
8,2021-01-31 20:57:55.804,18.68,4.30,Shallow,Mild
9,2021-01-31 20:54:42.248,75.53,4.20,Intermediate,Mild


In [ ]:
df.to_csv("earthquakes.csv", index=False)
print("CSV saved successfully!")


CSV saved successfully!


In [ ]:
from google.colab import files
files.download("earthquakes.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>